# 02 Core Analysis

Core meta questions on cleaned TFT data: costs, traits, items, units, and winner identifiers.

In [ ]:
# Setup
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

MARKERS = {"requirements.txt", "Projectplan.md", ".git"}
PROJECT_ROOT = None
cwd = Path.cwd()
for path in [cwd, *cwd.parents]:
    if any((path / m).exists() for m in MARKERS):
        PROJECT_ROOT = path
        break
if PROJECT_ROOT is None:
    PROJECT_ROOT = cwd

DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
CLEANED_DIR = DATA_PROCESSED / 'cleaned'
CANONICAL_DIR = DATA_PROCESSED / 'canonical_original'

OUTPUTS_DIR = PROJECT_ROOT / 'outputs'
FIGURES_DIR = OUTPUTS_DIR / 'figures'
TABLES_DIR = OUTPUTS_DIR / 'tables'
for p in [OUTPUTS_DIR, FIGURES_DIR, TABLES_DIR]:
    p.mkdir(parents=True, exist_ok=True)

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 180)
plt.rcParams.update({'figure.figsize': (10, 6), 'figure.dpi': 120})
sns.set_theme(style='whitegrid', palette='muted')


In [ ]:
# Load data
participants = pd.read_csv(CLEANED_DIR / 'participants.csv')
traits = pd.read_csv(CLEANED_DIR / 'traits.csv')
units = pd.read_csv(CLEANED_DIR / 'units.csv')
units_ref = pd.read_csv(CANONICAL_DIR / 'units_s16.csv')
traits_ref = pd.read_csv(CANONICAL_DIR / 'traits_s16.csv')
items_ref = pd.read_csv(CANONICAL_DIR / 'items_s16.csv')

# Normalize strings
for df, cols in [
    (units, ['unit_name','item_0','item_1','item_2']),
    (units_ref, ['name']),
    (traits, ['trait_id']),
    (traits_ref, ['name_corrected','name']),
    (items_ref, ['name','comp_0','comp_1'])
]:
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()

for df in [participants, units, traits]:
    for c in ['match_id','puuid']:
        if c in df.columns:
            df[c] = df[c].astype(str)

# Winners (top4)
winners = participants[participants['placement'] <= 4]
winner_ids = winners[['match_id','puuid']]
units_win = units.merge(winner_ids, on=['match_id','puuid'], how='inner')
traits_win = traits.merge(winner_ids, on=['match_id','puuid'], how='inner')

# Maps and valid sets
unit_cost_map = dict(zip(units_ref['name'], units_ref['cost']))
trait_key = traits_ref['name_corrected'] if 'name_corrected' in traits_ref.columns else traits_ref['name']
trait_unique_series = traits_ref['is_unique'] if 'is_unique' in traits_ref.columns else pd.Series([False]*len(trait_key))
trait_unique_map = dict(zip(trait_key, trait_unique_series))
trait_display_map = dict(zip(trait_key, traits_ref['name']))
valid_items = set(items_ref['name']).union(items_ref.get('comp_0', pd.Series())).union(items_ref.get('comp_1', pd.Series()))


## Q1: Cost distribution for winners

In [ ]:
units_cost_all = units.copy()
units_cost_all['unit_cost'] = units_cost_all['unit_name'].map(unit_cost_map)
units_cost_win = units_win.copy()
units_cost_win['unit_cost'] = units_cost_win['unit_name'].map(unit_cost_map)

cost_counts_all = units_cost_all.groupby(['match_id','puuid','unit_cost']).size().unstack(fill_value=0)
cost_counts_win = units_cost_win.groupby(['match_id','puuid','unit_cost']).size().unstack(fill_value=0)
cost_counts_mean_all = cost_counts_all.mean().sort_index()
cost_counts_mean_win = cost_counts_win.mean().sort_index()

cost_df = (
    pd.concat([cost_counts_mean_all.rename('avg_all'), cost_counts_mean_win.rename('avg_win')], axis=1)
    .fillna(0)
)
cost_df = cost_df.reset_index().rename(columns={'unit_cost':'cost'})
cost_df.to_csv(TABLES_DIR / '02_unit_cost_distribution.csv', index=False)

print('Average units per cost (all players):')
print(cost_counts_mean_all)
print('Average units per cost (winners):')
print(cost_counts_mean_win)

cost_df_sorted = cost_df.sort_values('cost')

plt.figure(figsize=(8,5))
plot_cost = cost_df_sorted.melt(id_vars='cost', value_vars=['avg_all','avg_win'], var_name='group', value_name='avg_units')
ax = sns.barplot(data=plot_cost, x='cost', y='avg_units', hue='group')
plt.title('Avg units per cost (all vs winners)')
plt.xlabel('Unit cost')
plt.ylabel('Average units on board')
plt.tight_layout()
plt.savefig(FIGURES_DIR / '02_cost_distribution_all_vs_winners.png', bbox_inches='tight')
plt.close()


In [ ]:
# Unit popularity by cost
units_cost_all = units.copy()
units_cost_all['unit_cost'] = units_cost_all['unit_name'].map(unit_cost_map)
units_cost_win = units_win.copy()
units_cost_win['unit_cost'] = units_cost_win['unit_name'].map(unit_cost_map)

unit_freq_all = units_cost_all.groupby(['unit_cost','unit_name']).size().rename('count_all').reset_index()
unit_freq_win = units_cost_win.groupby(['unit_cost','unit_name']).size().rename('count_win').reset_index()
unit_freq = (unit_freq_all
    .merge(unit_freq_win, on=['unit_cost','unit_name'], how='left')
    .fillna(0)
)
unit_freq['win_share'] = (unit_freq['count_win'] / unit_freq['count_all'].replace(0, np.nan)).fillna(0)
unit_freq = unit_freq.sort_values(['unit_cost','count_all'], ascending=[True, False])
unit_freq.to_csv(TABLES_DIR / '02_units_popularity_by_cost.csv', index=False)

top_units = unit_freq.groupby('unit_cost').head(5)
print('Top units by cost (all players):')
print(top_units[['unit_cost','unit_name','count_all','count_win','win_share']])

g = sns.catplot(data=top_units, x='unit_name', y='count_all', col='unit_cost', col_wrap=3, kind='bar', sharex=False, sharey=False, height=4)
g.set_titles('Cost {col_name}')
g.set_axis_labels('Unit', 'Count (all)')
for ax in g.axes.flatten():
    ax.tick_params(axis='x', rotation=70)
plt.tight_layout()
plt.savefig(FIGURES_DIR / '02_units_popularity_by_cost.png', bbox_inches='tight')
plt.close()


## Q2: Item popularity (counts, winners vs all)

In [ ]:
item_cols = [c for c in units.columns if c.startswith('item_')]
items_long = units.melt(id_vars=['match_id','puuid','unit_name'], value_vars=item_cols, var_name='slot', value_name='item')
items_long = items_long.dropna(subset=['item'])
items_long['item'] = items_long['item'].astype(str).str.strip()
items_long = items_long[items_long['item'].isin(valid_items)]
items_long_win = items_long.merge(winner_ids, on=['match_id','puuid'], how='inner')

item_freq_all = items_long['item'].value_counts().reset_index()
item_freq_all.columns = ['item','count_all']
item_freq_win = items_long_win['item'].value_counts().reset_index()
item_freq_win.columns = ['item','count_win']

item_freq = item_freq_all.merge(item_freq_win, on='item', how='outer').fillna(0)
item_freq = item_freq.sort_values('count_all', ascending=False)
item_freq.to_csv(TABLES_DIR / '02_items_frequency.csv', index=False)

emblem_mask = item_freq['item'].str.contains('Emblem', case=False, na=False)
emblem_freq = item_freq[emblem_mask]

top_items = item_freq.head(15)
print('Top items (all):')
print(top_items)
print('Top items (winners):')
print(item_freq.sort_values('count_win', ascending=False).head(15))
print('Emblems (all vs winners):')
print(emblem_freq.sort_values('count_all', ascending=False).head(10))

plt.figure(figsize=(10, 5))
plot_df = top_items.melt(id_vars='item', value_vars=['count_all','count_win'], var_name='group', value_name='count')
ax = sns.barplot(data=plot_df, x='item', y='count', hue='group')
half_points = top_items['count_all'] / 2
ax.scatter(ax.get_xticks(), half_points, color='red', marker='o', label='Half of all')
plt.title('Top item frequencies')
plt.xlabel('Item')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR / '02_items_popularity.png', bbox_inches='tight')
plt.close()


## Q3: Traits popularity and power

In [ ]:
# Trait popularity using active traits (tier_current > 0)
traits_with_meta = traits.copy()
traits_with_meta['is_unique'] = traits_with_meta['trait_id'].map(trait_unique_map)
traits_with_meta['trait_name'] = traits_with_meta['trait_id'].map(trait_display_map).fillna(traits_with_meta['trait_id'])

traits_win_meta = traits_win.copy()
traits_win_meta['is_unique'] = traits_win_meta['trait_id'].map(trait_unique_map)
traits_win_meta['trait_name'] = traits_win_meta['trait_id'].map(trait_display_map).fillna(traits_win_meta['trait_id'])

traits_filtered = traits_with_meta[traits_with_meta['tier_current'] > 0]
traits_win_filtered = traits_win_meta[traits_win_meta['tier_current'] > 0]

trait_all_no_excl = traits_filtered[~traits_filtered['is_unique'].fillna(False)]['trait_name'].value_counts()
trait_win_no_excl = traits_win_filtered[~traits_win_filtered['is_unique'].fillna(False)]['trait_name'].value_counts()

print('Top traits (winners, active, no exclusive):')
print(trait_win_no_excl.head(15))
print('Top traits (all, active, no exclusive):')
print(trait_all_no_excl.head(15))

trait_freq = (
    trait_all_no_excl.reset_index().rename(columns={'trait_name':'trait_name','count':'count_all'})
    .merge(trait_win_no_excl.reset_index().rename(columns={'trait_name':'trait_name','count':'count_win'}), on='trait_name', how='outer')
    .fillna(0)
    .sort_values('count_all', ascending=False)
    .head(15)
)
trait_freq['trait_id'] = trait_freq['trait_name']
trait_freq = trait_freq[['trait_id','trait_name','count_all','count_win']]
trait_freq.to_csv(TABLES_DIR / '02_traits_frequency_no_exclusive.csv', index=False)

order_traits = trait_freq['trait_name']

plt.figure(figsize=(10, 5))
plot_trait = trait_freq.melt(id_vars=['trait_name'], value_vars=['count_all','count_win'], var_name='group', value_name='count')
ax = sns.barplot(data=plot_trait, x='trait_name', y='count', hue='group', order=order_traits)
half_trait = trait_freq['count_all'] / 2
ax.scatter(ax.get_xticks(), half_trait, color='red', marker='o', label='Half of all')
plt.title('Top traits (active, no exclusive)')
plt.xlabel('Trait')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR / '02_traits_popularity.png', bbox_inches='tight')
plt.close()


In [ ]:
# Highest active trait per winner (traits table) and fast9 split
traits_win_active = traits_win.copy()
traits_win_active['trait_name'] = traits_win_active['trait_id'].map(trait_display_map).fillna(traits_win_active['trait_id'])
traits_win_active['is_unique'] = traits_win_active['trait_id'].map(trait_unique_map)
traits_win_active = traits_win_active[(traits_win_active['tier_current'] > 0) & (~traits_win_active['is_unique'].fillna(False))]

# Pick highest-tier active trait per winner (tie-breaker alphabetical)
main_trait = (
    traits_win_active.sort_values(['match_id','puuid','tier_current','trait_name'], ascending=[True, True, False, True])
    .groupby(['match_id','puuid']).head(1)
)
main_trait_counts = main_trait['trait_name'].value_counts().reset_index(name='count').rename(columns={'index':'trait_name'})
main_trait_counts.to_csv(TABLES_DIR / '02_winner_main_traits.csv', index=False)

plt.figure(figsize=(10,5))
sns.barplot(data=main_trait_counts.head(15), x='trait_name', y='count')
plt.title('Highest active trait of winners')
plt.xlabel('Trait')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig(FIGURES_DIR / '02_winner_main_traits.png', bbox_inches='tight')
plt.close()

# Fast9 split (high-cost heavy)
units_win_cost = units_win.copy()
units_win_cost['unit_cost'] = units_win_cost['unit_name'].map(unit_cost_map)
player_costs = units_win_cost.groupby(['match_id','puuid']).agg(
    total_units=('unit_name','size'),
    high_cost_units=('unit_cost', lambda x: (x >= 5).sum())
).reset_index()
player_costs['high_cost_share'] = player_costs['high_cost_units'] / player_costs['total_units'].replace(0, 1)
player_costs['is_fast9'] = (player_costs['high_cost_units'] >= 4) | (player_costs['high_cost_share'] >= 0.4)

main_trait = main_trait.merge(player_costs[['match_id','puuid','is_fast9']], on=['match_id','puuid'], how='left').fillna({'is_fast9': False})
fast9_main = main_trait[main_trait['is_fast9']]
others_main = main_trait[~main_trait['is_fast9']]

others_main_counts = others_main['trait_name'].value_counts().reset_index(name='count').rename(columns={'index':'trait_name'})
others_main_counts.to_csv(TABLES_DIR / '02_winner_main_traits_non_fast9.csv', index=False)
if not others_main_counts.empty:
    plt.figure(figsize=(10,5))
    sns.barplot(data=others_main_counts.head(15), x='trait_name', y='count')
    plt.title('Highest active trait of winners (non-fast9)')
    plt.xlabel('Trait')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / '02_winner_main_traits_non_fast9.png', bbox_inches='tight')
    plt.close()
else:
    print('No non-fast9 winners found')

fast9_players = player_costs[player_costs['is_fast9']][['match_id','puuid']]
units_fast9 = units_win_cost.merge(fast9_players, on=['match_id','puuid'], how='inner')
units_fast9_nonhigh = units_fast9[units_fast9['unit_cost'] < 5]
fast9_unit_counts = units_fast9_nonhigh['unit_name'].value_counts().reset_index(name='count').rename(columns={'index':'unit_name'})
fast9_unit_counts.to_csv(TABLES_DIR / '02_fast9_non_highcost_units.csv', index=False)
if not fast9_unit_counts.empty:
    plt.figure(figsize=(10,5))
    sns.barplot(data=fast9_unit_counts.head(15), x='unit_name', y='count')
    plt.title('Fast9 players: non-high-cost units')
    plt.xlabel('Unit')
    plt.ylabel('Count')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(FIGURES_DIR / '02_fast9_non_highcost_units.png', bbox_inches='tight')
    plt.close()
else:
    print('No fast9 players found for non-high-cost unit plot')


## Q4: Popular/powerful non-5/7 cost units

In [ ]:
units_cost_win = units_cost_win.copy()
units_cost_win['unit_cost'] = units_cost_win['unit_name'].map(unit_cost_map)
units_cost_win_non_high = units_cost_win[units_cost_win['unit_cost'] < 5]

unit_counts_win_non_high = units_cost_win_non_high['unit_name'].value_counts()
print('Top non-5/7 cost units on winning boards:')
print(unit_counts_win_non_high.head(15))

three_star = units_win[units_win['unit_tier'] == 3]['unit_name'].value_counts()
print('Three-star units frequency (winners):')
print(three_star.head(15))

units_with_three = units_win[(~units_win['item_0'].isna()) & (~units_win['item_1'].isna()) & (~units_win['item_2'].isna())].copy()
units_with_three['unit_cost'] = units_with_three['unit_name'].map(unit_cost_map)
unit_three_items = units_with_three['unit_name'].value_counts()
print('Units with all three items (winners):')
print(unit_three_items.head(15))

unit_three_items_detail = (
    units_with_three
    .groupby(['unit_name','unit_tier','unit_cost'])
    .size()
    .reset_index(name='count')
    .sort_values('count', ascending=False)
)
unit_three_items_detail.to_csv(TABLES_DIR / '02_units_three_items_detail.csv', index=False)

print('Units with all three items by star level and cost (winners):')
print(unit_three_items_detail.head(15))


## Q5: High-cost units (>=5) performance and traits

In [ ]:
high_cost_units = units_cost_win[units_cost_win['unit_cost'] >= 5]
high_cost_counts = high_cost_units['unit_name'].value_counts()
print('High-cost unit frequency (winners):')
print(high_cost_counts.head(15))

# Traits from canonical units to avoid mismatches (non-unique only)
unit_trait_map = (
    units_ref[['name','trait']]
    .assign(trait=lambda df: df['trait'].astype(str).str.split(';'))
    .explode('trait')
)
unit_trait_map['trait'] = unit_trait_map['trait'].astype(str).str.strip()
unit_trait_map['trait_name'] = unit_trait_map['trait'].map(trait_display_map).fillna(unit_trait_map['trait'])
trait_unique_by_name = dict(zip(traits_ref['name'], traits_ref['is_unique']))
unit_trait_map['is_unique'] = unit_trait_map['trait_name'].map(trait_unique_by_name).fillna(False)
unit_trait_map = unit_trait_map[~unit_trait_map['is_unique']]

high_cost_traits = high_cost_units[['unit_name']].merge(unit_trait_map, left_on='unit_name', right_on='name', how='left')
trait_cofreq = (
    high_cost_traits.groupby(['unit_name','trait_name']).size().rename('count').reset_index()
    .sort_values('count', ascending=False)
)
trait_cofreq.to_csv(TABLES_DIR / '02_high_cost_trait_cofreq.csv', index=False)

print('Top co-occurring traits with high-cost units (canonical, non-unique):')
print(trait_cofreq.head(20))

# Active high-tier non-exclusive traits on board (tier > 1)
traits_win_meta = traits_win.copy()
traits_win_meta['is_unique'] = traits_win_meta['trait_id'].map(trait_unique_map)
non_exclusive_high = traits_win_meta[(~traits_win_meta['is_unique'].fillna(False)) & (traits_win_meta['tier_current'] > 1)]
trait_counts_per_player = non_exclusive_high.groupby(['match_id','puuid']).size().rename('non_excl_high_trait_count')
high_cost_units = high_cost_units.merge(trait_counts_per_player, on=['match_id','puuid'], how='left').fillna({'non_excl_high_trait_count':0})
high_cost_no_high_trait = high_cost_units[high_cost_units['non_excl_high_trait_count'] == 0]['unit_name'].value_counts()
high_cost_no_high_trait.to_csv(TABLES_DIR / '02_high_cost_units_no_high_traits.csv')
print('High-cost units on boards with no active high-tier non-exclusive traits:')
print(high_cost_no_high_trait)


## Q6: Key identifiers for winners vs losers

In [ ]:
participants = participants.assign(placement_score=9 - participants['placement'])

unit_counts = units.groupby(['match_id','puuid']).size().rename('units_per_board')
items_per_player = units.groupby(['match_id','puuid']).size().rename('num_of_items')
units_cost_win['unit_cost'] = units_cost_win['unit_name'].map(unit_cost_map)
units_cost_win['unit_total_cost'] = units_cost_win['unit_cost'] * units_cost_win['unit_tier']
unit_cost_avg = units_cost_win.groupby(['match_id','puuid'])['unit_cost'].mean().rename('avg_unit_cost')
unit_total_cost_avg = units_cost_win.groupby(['match_id','puuid'])['unit_total_cost'].mean().rename('avg_unit_total_cost')
avg_trait_tier = traits.groupby(['match_id','puuid'])['tier_current'].mean().rename('avg_trait_tier')

player_df = (
    participants
    .merge(unit_counts, on=['match_id','puuid'], how='left')
    .merge(items_per_player, on=['match_id','puuid'], how='left')
    .merge(unit_cost_avg, on=['match_id','puuid'], how='left')
    .merge(unit_total_cost_avg, on=['match_id','puuid'], how='left')
    .merge(avg_trait_tier, on=['match_id','puuid'], how='left')
)

corr_cols = ['placement_score','level','num_of_items','avg_trait_tier','avg_unit_cost','avg_unit_total_cost']
corr_matrix = player_df[corr_cols].corr()
print(corr_matrix)

features = player_df.copy()
features['is_win_flag'] = features['placement_score'] >= 5
summary = features.groupby(features['is_win_flag']).agg({
    'placement':'mean',
    'placement_score':'mean',
    'level':'mean',
    'num_of_items':'mean',
    'avg_trait_tier':'mean',
    'avg_unit_cost':'mean',
    'avg_unit_total_cost':'mean'
}).rename(index={True:'winners', False:'others'})
print(summary)
